In [133]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 


In [134]:
## include results 

results = pd.read_csv("random_results.xlsx")

results_noise0 = results[["group", "size", "size_class", "vars", "correct"]]
results_noise0['correct'] = results_noise0['correct'].apply(lambda x: 100 * x)

results_noise_0001 = results[["group", "size", "size_class", "correct.1"]]
results_noise_0001['correct.2'] = results_noise_0001['correct.1'].apply(lambda x: 100 * x)

results_noise_001 = results[["group", "size", "size_class", "correct.2"]]
results_noise_001['correct.2'] = results_noise_001['correct.2'].apply(lambda x: 100 * x)


/tmp/ipykernel_4394/884508441.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_noise0['correct'] = results_noise0['correct'].apply(lambda x: 100 * x)
/tmp/ipykernel_4394/884508441.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_noise_0001['correct.2'] = results_noise_0001['correct.1'].apply(lambda x: 100 * x)
/tmp/ipykernel_4394/884508441.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [135]:
############################# real instances (groundtruth): AI_ Feynman first #############################
ALGORITHMS = ["AIFeynman", "Operon", "AFP_FE", "AFP", "BSR", "DSR", "EPLEX", "FEAT", "FFX", "GP-GOMEA", "ITEA", "MRGP", "SBP-GP", "gplearn"   ]
data_groups = ["Feynman", "Strogatz"]
noise_levels = [0.0, 0.001, 0.01 ]

## index of groups
data_groups_index = 1 # data_groups[ data_groups_index ]
noise_levels_index = 0 # noise_levels[ noise_levels_index ]
criterion_column = "accuracy_solution" #"symbolic_solution_rate_(%)" #accuracy_solution
criterion_feynman_column = "R2>0.999"# correct
################# dataset ####################################

results_groundtruth = pd.read_csv("groundtruth.csv")
results_groundtruth_noise = results_groundtruth[(results_groundtruth["target_noise"] == noise_levels[ noise_levels_index ]) & (results_groundtruth["data_group"] == data_groups[ data_groups_index ])]

########################## aggregate ##################################################################################################
results_groundtruth_noise_groupby_alg = results_groundtruth_noise.groupby(["algorithm"]).aggregate({ criterion_column : "mean" }) # symbolic_solution_rate_(%)
results_groundtruth_noise_groupby_alg_ri = results_groundtruth_noise_groupby_alg.reset_index()
results_groundtruth_noise_groupby_alg_ri

results_groundtruth_noise_required = results_groundtruth_noise[["algorithm", "dataset", criterion_column]]
results_groundtruth_noise_required

########## feynman or stogatz results ==> include ##########################

def feynman_or_strogatz(filename, noise_level):
    
    file_name = ""
    if filename == "Feynman":
        file_name = "feynman_results.csv"
    else:
        file_name = "strogatz_results.csv"
    
    feynman_results = pd.read_csv(file_name, delimiter=",", encoding='utf-16') #, encoding='cp1252')
    print(feynman_results)
    feynman_results_groupby = feynman_results.groupby(["instance"]).aggregate({criterion_feynman_column : "sum", criterion_feynman_column +".1" : "sum", criterion_feynman_column + ".2" : "sum" })
    feynman_results_groupby_ri = feynman_results_groupby.reset_index()
    feynman_results_groupby_ri['algorithm'] = feynman_results_groupby_ri.apply(lambda x: 'Rils-Rols', axis = 1)

    if noise_level == 0.0: 
        feynman_results_groupby_ri = feynman_results_groupby_ri[["algorithm", "instance", criterion_feynman_column]]
    elif noise_level == 0.001:
        feynman_results_groupby_ri = feynman_results_groupby_ri[["algorithm", "instance", criterion_feynman_column + ".1"]]
    else:
        feynman_results_groupby_ri = feynman_results_groupby_ri[["algorithm", "instance", criterion_feynman_column + ".2"]]
    
    feynman_results_groupby_ri.columns = ["algorithm", "dataset",  criterion_feynman_column]
    
    if criterion_column != "accuracy_solution":
        feynman_results_groupby_ri[criterion_feynman_column] = feynman_results_groupby_ri[ criterion_feynman_column ].apply(lambda x: x * 10 )
    else:
        feynman_results_groupby_ri[criterion_feynman_column] = feynman_results_groupby_ri[ criterion_feynman_column ].apply(lambda x: x / 10 )

    return feynman_results_groupby_ri

#results_groundtruth_noise_required

####################################### Strogatz and Feynman dataset include for Rils-Rols #################################################
feynman_results_noise = pd.DataFrame()

if data_groups[ data_groups_index ] == "Feynman": 
    feynman_results_noise = feynman_or_strogatz(data_groups[0], noise_levels[ noise_levels_index ])
else:
    feynman_results_noise = feynman_or_strogatz(data_groups[1], noise_levels[ noise_levels_index ])

##################### merge datasets columns: algorithms ==> rows dataset: symbolc_sol. rate ##########################################################

results_groundtruth_noise_required

########## results_groundtruth_noise_required: split by algorithm ###################################
results_groundtruth_by_algorithm = {}

for algorithm in ALGORITHMS:
    results_groundtruth_noise_alg = results_groundtruth_noise_required[results_groundtruth_noise_required["algorithm"] == algorithm]
    results_groundtruth_by_algorithm[algorithm] = results_groundtruth_noise_alg
    results_groundtruth_by_algorithm[algorithm ].drop(columns = ["algorithm"], axis = 1, inplace=True)
    results_groundtruth_by_algorithm[algorithm].columns = ["dataset", algorithm ]


             instance   seed                        true model  test R2  \
0    strogatz_bacres1    860   20 - x - (x * y)/(1+0.5 * x**2)      1.0   
1    strogatz_bacres1   4426   20 - x - (x * y)/(1+0.5 * x**2)      1.0   
2    strogatz_bacres1   5390   20 - x - (x * y)/(1+0.5 * x**2)      1.0   
3    strogatz_bacres1  14423   20 - x - (x * y)/(1+0.5 * x**2)      1.0   
4    strogatz_bacres1  15795   20 - x - (x * y)/(1+0.5 * x**2)      1.0   
..                ...    ...                               ...      ...   
135     strogatz_vdp2  16850                     -(1)/(10) * x      1.0   
136     strogatz_vdp2  21962                     -(1)/(10) * x      1.0   
137     strogatz_vdp2  23654                     -(1)/(10) * x      1.0   
138     strogatz_vdp2  28020                     -(1)/(10) * x      1.0   
139     strogatz_vdp2  29910                     -(1)/(10) * x      1.0   

     test RMSE  size   time                           model  R2>0.999  \
0          0.0    16  121.

/tmp/ipykernel_4394/3673740932.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_groundtruth_by_algorithm[algorithm ].drop(columns = ["algorithm"], axis = 1, inplace=True)
/tmp/ipykernel_4394/3673740932.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_groundtruth_by_algorithm[algorithm ].drop(columns = ["algorithm"], axis = 1, inplace=True)
/tmp/ipykernel_4394/3673740932.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

In [136]:
########## Feynamn/Strogatz: reduction
print(feynman_results_noise)

data_all_results = pd.DataFrame()
feynman_results_noise.drop(columns = ["algorithm"], axis = 1, inplace=True)
feynman_results_noise.columns = ["dataset", "Rils-Rols"]


    algorithm              dataset  R2>0.999
0   Rils-Rols     strogatz_bacres1       1.0
1   Rils-Rols     strogatz_bacres2       1.0
2   Rils-Rols     strogatz_barmag1       0.6
3   Rils-Rols     strogatz_barmag2       1.0
4   Rils-Rols     strogatz_glider1       1.0
5   Rils-Rols     strogatz_glider2       1.0
6   Rils-Rols         strogatz_lv1       1.0
7   Rils-Rols         strogatz_lv2       1.0
8   Rils-Rols   strogatz_predprey1       0.9
9   Rils-Rols   strogatz_predprey2       1.0
10  Rils-Rols  strogatz_shearflow1       0.1
11  Rils-Rols  strogatz_shearflow2       1.0
12  Rils-Rols        strogatz_vdp1       1.0
13  Rils-Rols        strogatz_vdp2       1.0


In [137]:
feynman_results_noise.mean()

/tmp/ipykernel_4394/3733515722.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  feynman_results_noise.mean()


Rils-Rols    0.9
dtype: float64

In [138]:
### MERGE:
data_all_results = feynman_results_noise.copy()
### merge results
for algorithm in ALGORITHMS:
    data_all_results = data_all_results.merge(results_groundtruth_by_algorithm[algorithm], on = ["dataset"])   
#### save to .csv
data_all_results['noise'] = data_all_results.apply(lambda x: noise_levels[ noise_levels_index ], axis = 1) 
data_all_results.to_csv( data_groups[ data_groups_index ]  + "_noise_" +  str(noise_levels[ noise_levels_index ] ).replace( ".", "_" ) + criterion_column + ".csv", sep=",")

print(data_groups[ data_groups_index ]  + "_noise_" +  str(noise_levels[ noise_levels_index ] ).replace( ".", "_" ) + ".csv")
 

Strogatz_noise_0_0.csv


In [139]:
data_all_results.mean()

/tmp/ipykernel_4394/3284405129.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data_all_results.mean()


Rils-Rols    0.900000
AIFeynman    0.357143
Operon       0.928571
AFP_FE       0.285714
AFP          0.250000
BSR          0.214286
DSR          0.142857
EPLEX        0.214286
FEAT         0.428571
FFX          0.000000
GP-GOMEA     0.714286
ITEA         0.214286
MRGP         0.892857
SBP-GP       0.785714
gplearn      0.071429
noise        0.000000
dtype: float64

In [140]:
data_all_results['Rils-Rols'].mean()

0.9

In [141]:
#### statistics 
from scipy import stats
print(data_all_results['Rils-Rols'].mean())
print(data_all_results['AFP_FE'].mean())

###  the Wilcoxon signed rank test
#here was a statistically significant difference if  p < 0.05; otherwise not 
stats.wilcoxon(data_all_results['Rils-Rols'], data_all_results['AFP_FE'])## reject hypothesis 



0.9
0.2857142857142857


/home/marko/.local/lib/python3.8/site-packages/scipy/stats/_morestats.py:3145: UserWarning: Exact p-value calculation does not work if there are ties. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


WilcoxonResult(statistic=0.0, pvalue=0.0028110405915931177)

In [142]:
#f_s_data = pd.read_csv("feynman_strogatz_noise0_001.csv")
#
#stats.ranksums(f_s_data["Rils_Rols"], f_s_data["DSR"])

In [143]:
#f_s_data.head(50)